<a href="https://colab.research.google.com/github/solly29/Google-colab/blob/master/%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!ls ml-latest-small.zip

--2020-04-09 09:24:02--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.1s    

2020-04-09 09:24:02 (8.02 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

ml-latest-small.zip


In [2]:
!unzip ml-latest-small.zip
!ls ml-latest-small/

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  
links.csv  movies.csv  ratings.csv  README.txt	tags.csv


In [0]:
from pathlib import Path
import pandas as pd # 파이썬에서 사용하는 데이터분석 라이브러리이다.
import numpy as np

In [4]:
data = pd.read_csv('./ml-latest-small/ratings.csv') # 트레이닝 데이터를 가지고온다.
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# **데이터 인코딩**

In [5]:
# 먼저 데이터를 train과 validation데이터로 나눈다.
np.random.seed(3) # 난수 생성 알고리즘 seed
msk = np.random.rand(len(data)) < 0.8 # 난수를 생성한다(0~1사이) 이때 0.8보다 작으면 true
train = data[msk].copy()              # msk에서 true인 값만 복사한다.
val = data[~msk].copy()               # ~는 반전 연산자이다.
print(train.head())
print(val.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
6       1      101     5.0  964980868
    userId  movieId  rating  timestamp
4        1       50     5.0  964982931
5        1       70     3.0  964982400
29       1      543     4.0  964981179
30       1      552     4.0  964982653
32       1      590     4.0  964982546


In [0]:
# 다음은 Pandas의 컬럼을 범주형의 id로 인코드해주는 함수이다
def proc_col(col, train_col=None):
    """ Encodes a pandas column with continous ids. """
    # train column에서 유일한 row를 찾는다(중복을 제거한다.) 즉 사용자 혹은 영화이다
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    # 사용자/영화를 인덱스와 매핑해준다(딕셔너리로 만든다.)
    # 인덱스 번호과 원소를 튜플 형태로 뽑아서 딕셔너리에 원소: 인덱스로 저장한다.
    name2idx = {o:i for i,o in enumerate(uniq)}
    # 그리고 그것을 포맷팅해서 리턴한다
    # 반환할때 매핑한 dic과 numpy배열과 영화/사용자의 수를 반환한다.
    # numpy는 인수로 받은 칼럼의 값을 name2idx의 키로 넣고 value를 배열로 저장한다.
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

In [0]:
# 다음은 실제로 데이터를 인코딩으로 만들어주는 함수이다
# 위에서 정의해준 proc_col을 사용한다
def encode_data(df, train=None):
    """ Encodes rating data with continous user and movie ids. 
    If train is provided, encodes df with the same encoding as train.
    """
    df = df.copy()
    for col_name in ["userId", "movieId"]:
        train_col = None
        if train is not None:
            train_col = train[col_name] # train_ol에 userid나 movield의 column을 넣는다. proc_col에 teain column을 넘겨줘야된다.
        _,col,_ = proc_col(df[col_name], train_col) # 다른값은 무시하고 배열만 받아온다.
        df[col_name] = col              # 인덱스와 배핑한 값으로 바꿔준다.
        df = df[df[col_name] >= 0]      # df[col_name]의 값이 -1이면 영화나 사용자가 없다는 뜻이다. 그래서 아예 그 행을 false로 만드는것 같다.
    return df

In [8]:
# Test와 Validation인코딩을 만들어준다
# 사용자나 영화에게 순서(숫자)를 부여해준다.
df_train = encode_data(train)
df_val = encode_data(val, train)
print(df_train.head())
print(df_val.head())

   userId  movieId  rating  timestamp
0       0        0     4.0  964982703
1       0        1     4.0  964981247
2       0        2     4.0  964982224
3       0        3     5.0  964983815
6       0        4     5.0  964980868
    userId  movieId  rating  timestamp
4        0      388     5.0  964982931
5        0      995     3.0  964982400
29       0      841     4.0  964981179
30       0      567     4.0  964982653
32       0      402     4.0  964982546


# **임베딩 레이어**
임베딩이란?  
범주형 자료를 연속형 백터 형태로 변환시키는 것  
**word Embedding**  
자연어를 R차원의 백터로 매핑시켜주는 것. 예를들어 cat이나 mat같은 단어를 특정 차원의 백터로 바꾸어주는 것이다.

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
# 아래의 임베딩모델은 최대 10명의 사용자나 3개의 아이템에 대한 관계를 표현한다
# 임베딩의 숫자들은 랜덤으로 초기화 된다
embed = nn.Embedding(10,3) # 10*3의 백터를 만든다. 
# 10명까지의 id이니 6개를 넣어준다
a = torch.LongTensor([[1,2,0,4,5,1]])
embed(a)

tensor([[[-0.8929,  0.1370, -0.8028],
         [-2.0023,  0.3582, -0.4300],
         [ 0.0294, -0.6888,  0.1922],
         [ 1.0960,  0.6110,  0.3892],
         [-0.3280,  0.1259,  0.6770],
         [-0.8929,  0.1370, -0.8028]]], grad_fn=<EmbeddingBackward>)

# **행렬분해 모델**
임베딩 두개의 행 즉 벡터들의 각각 내적곱을 하는 역할을 하는 모델이다.

In [0]:
class MatrixFactorization(nn.Module):
  # init은 생성자와 같다. 메소드이름 양옆에 _가 있으면 외부에서 사용하지 말라는 뜻이다.
  # __는 약속되어있는 메소드이다.(아마)
  # 파이썬은 메서드에 첫 번째 인자는 항상 self이어야되고 파이썬이 자동으로 넘겨준다.
  # self는 자기 자신 인스턴스이다.
  def __init__(self, num_users, num_items, emb_size=100):
    super().__init__() # 부모 생성자 호출
    self.user_emb = nn.Embedding(num_users, emb_size)
    self.item_emb = nn.Embedding(num_items, emb_size)
    #  정규분포를 사용해 임베딩을 초기화한다
    #  입베드 가중치 초기화(최소 0, 최대 0.05)
    self.user_emb.weight.data.uniform_(0, 0.05)
    self.item_emb.weight.data.uniform_(0, 0.05)
  
  # 여기서 내적곱을 해준다.
  def forward(self, u, v):
    u = self.user_emb(u)
    v = self.item_emb(v)
    return (u*v).sum(1)

밑의 내용은 위의 행렬분해 모델이 하는일을 절차적으로 작성한것이다.  
밑의 내용은 학습단계가 없는 절차이다.

In [12]:
# 임의의 테스트 데이터를 만들어본다
# 사용자는 6명 영화는 3개로 이루어져있다
# 사용자와 영화 평점을 각 행렬에 넣고 Pandas DF를 생성
users = [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6]
movies = [0, 1, 1, 2, 0, 1, 0, 3, 0, 3, 3, 1, 3]
ratings = [4, 5, 5, 3, 4, 4, 5, 2, 1, 4, 5, 1, 3]
columns = ['userId', 'movieId', 'rating']

# 행렬에서 a.T를 하면 행, 열이 바뀐다.
test_df = pd.DataFrame(np.asarray([users, movies, ratings]).T, columns=columns)
test_df

,userId,movieId,rating
0,0,0,4
1,0,1,5
2,1,1,5
3,1,2,3
4,2,0,4
5,2,1,4
6,3,0,5
7,3,3,2
8,4,0,1
9,4,3,4


In [13]:
num_users = len(test_df.userId.unique()) # 사용자 id에서 유일한 값을 찾는다.
num_items = len(test_df.movieId.unique())
emb_size = len(test_df.columns)          # 전체 열수로 임베드 레이어 사이즈를 정함
print("num_users: {}, num_items: {}, emb_size: {}".format(num_users, num_items, emb_size))

users = torch.LongTensor(test_df.userId.values) # 사용자 id로 long형 텐서를 만든다.(1차원)
items = torch.LongTensor(test_df.movieId.values)
user_emb = nn.Embedding(num_users, emb_size)    # 사용자의 수로 임베딩한다.
item_emb = nn.Embedding(num_items, emb_size)
print("embeddings: {}, {}".format(user_emb,item_emb))

U = user_emb(users)
V = item_emb(items)

# 임베딩의 각 행을 Dot Product해준다
# 아래의 텐서는 각 행의 결과이니 총 13개가 된다
(U * V).sum(1)

num_users: 7, num_items: 4, emb_size: 3
embeddings: Embedding(7, 3), Embedding(4, 3)


tensor([ 0.1816,  2.6892,  4.2526, -2.0913,  0.7216, -0.1684, -1.3953,  0.7173,
        -1.4833,  1.1095,  0.9377, -1.1713, -0.6194], grad_fn=<SumBackward1>)

# **행렬분해 모델 학습**
이제 실제 모델 클래스를 사용해서 행렬분해모델을 학습시킨다. 아래의 예제에서는 임베딩의 사이즈를 100개로 했는데, 이것은 사용자들과 영화들의 관계 즉 평점정보에서 100개의 특성을 뽑아낸다는 것의 의미이다.

In [25]:
# 실제 학습 데이터에서 사용자와 영화의 갯수를 구한다.
num_users = len(df_train.userId.unique())
num_items = len(df_train.movieId.unique())

num_users, num_items

(610, 8998)

In [26]:
# 행렬분해 모델을 만든다
# 임베딩(특성)의 갯수는 100개로 한다
model = MatrixFactorization(num_users, num_items, emb_size=100)
model

asas


MatrixFactorization(
  (user_emb): Embedding(610, 100)
  (item_emb): Embedding(8998, 100)
)

In [0]:
# 결과 로스
def validation_loss(model, unsqueeze=False):
  model.eval()   # 모델을 불러와서 테스트하는 용도이다. model.train은 학습용
  users = torch.LongTensor(df_val.userId.values)
  items = torch.LongTensor(df_val.movieId.values)
  ratings = torch.FloatTensor(df_val.rating.values)
  if unsqueeze:
    ratings = ratings.unsqueeze(1)   # unsqueeze()는 특정 위치에 1인 차원을 추가한다. 즉 1차원이 2차원으로 된다. 그 반대도 있다.
  y_hat = model(users, items)
  loss = F.mse_loss(y_hat, ratings)
  print("validation loss {:.3f}".format(loss.item()))

In [0]:
def train_mf(model, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
    # epochs는 몇번 돌릴것인가 이고 밑에건 모델을 최적화 시키는 것이다.
    # lr은 learning rate로 경사하강법에서 이것을 곱해서 다음지점(가중치)를 구한다.
    # learning rate를 너무 크지도 작지도 않은 적절한 값으로 해야 학습이 잘된다.
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    model.train()
    for i in range(epochs):
        users = torch.LongTensor(df_train.userId.values)
        items = torch.LongTensor(df_train.movieId.values)
        ratings = torch.FloatTensor(df_train.rating.values)
        if unsqueeze:
            ratings = ratings.unsqueeze(1)
        y_hat = model(users, items) # 순전파 단계(여기서 ratings을 계산(예측))
        loss = F.mse_loss(y_hat, ratings) # 실제 ratings와 비교하여 loss를 계산
        # 역전파 단계를 하기 전에 가중치들(매개변수)을 0을 초기화 시킨다. 
        # 왜냐하면 backward()를 하면 가중치가 누적이 되기떄문이다.
        optimizer.zero_grad()
        loss.backward()             # 역전파 단계
        optimizer.step()            # 매개변수 갱신
        print(loss.item()) 
    validation_loss(model, unsqueeze)  # 학습된 모델에서 loss를 출력한다.

순전파 - input 값이 은닉층을 지나며 가중치를 계산하여 마지막에 output 결과를 만들어 내는 것  
역전파 - 결과 값을 통해서 다시 역으로 input 방향으로 오차를 다시 보내며 가중치를 재업데이트 하는 것  
  
순전파의 결과값에서 오차(error)가 나오는데 이 오차를 다시 역방향으로 보내면서 가중치를 계산하면서 오차를 적용시킨다. 결과에 많이 미친 노드에 다 많은 오차를 돌려준다.  
  
weight decay는 오버피팅을 억제하는 방법중 하나이다.  
오버피팅은 모델이 너무 훈련 데이터에만 지나치게 적응하여 시험데이터에 제대로 반응하지 않는 현상이다. 이 오버피팅은 가중치값이 커서 발생하는 경우가 많다. weight decay의 값이 커질수록 가중치의 값이 작아지게되고 오버피팅 현상을 해소할수 있다.

In [29]:
train_mf(model, epochs=10, lr=0.1)

aaaa
12.908639907836914
aaaa
4.843044281005859
aaaa
2.6098411083221436
aaaa
3.085411787033081
aaaa
0.8479647636413574
aaaa
1.8237531185150146
aaaa
2.655244827270508
aaaa
2.132136583328247
aaaa
1.0890127420425415
aaaa
0.9774454236030579
aaaa
validation loss 1.849


In [19]:
train_mf(model, epochs=15, lr=0.01)

1.64070463180542
1.0032434463500977
0.7114315032958984
0.6612184643745422
0.7262377142906189
0.8042191863059998
0.8434268236160278
0.8348641395568848
0.7922900319099426
0.7366553544998169
0.6869567036628723
0.6552647352218628
0.6446753144264221
0.6500727534294128
0.6615650057792664
validation loss 0.821


In [20]:
train_mf(model, epochs=15, lr=0.01)

0.6693838238716125
0.6311591267585754
0.6390969157218933
0.6144055128097534
0.6053226590156555
0.6138280630111694
0.6117620468139648
0.5969518423080444
0.5848297476768494
0.5829227566719055
0.5840951204299927
0.5793365836143494
0.5687278509140015
0.5583487153053284
0.5519223213195801
validation loss 0.758
